In [1]:
# General libraries:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Personal libraries:
from robin_helperfunctions import round_to_hour
from robinhoodstrategy import cryptoStrategy, BackcastStrategy, Trade

# Deep learning libraries:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

In [2]:
# Load historical data from file:
df = pd.read_csv('eth_historical_4hrs.csv')

# Convert timezones and reformat:
df['time_period_start'] = pd.to_datetime(df['time_period_start']).dt.tz_convert('US/Central').dt.strftime('%m-%d-%Y %H:%M')

# Pull just the value fields:
df = df[['price_close', 'price_open', 'price_high', 'price_low', 'volume_traded', 'trades_count']]

# Load just ~2018 onwards:
# df = df[2500:]

df.head()

,price_close,price_open,price_high,price_low,volume_traded,trades_count
0,11.8895,11.87000,11.90000,11.85972,1076.171006,48
1,12.0000,11.89000,12.00000,11.89000,6184.443260,182
2,12.0300,12.00000,12.03000,11.93356,3391.487460,120
3,12.2000,12.02591,12.20000,12.00000,1590.240506,53
4,12.4310,12.20000,12.54799,12.20000,4801.330298,164


In [30]:
def entry_flag(df, lookforward_num):
    
    '''If there will be over a 5% gain in the next x periods, flag as a 1. Else 0.'''
    
    df['pctchange'] = df[['price_close']].pct_change(lookforward_num)
    
    entry_flags = []
    for i in range(len(df) - lookforward_num):
        if df.iloc[i+lookforward_num]['pctchange'] >= 0.03:
            entry_flags.append(2)
        elif 0 < df.iloc[i+lookforward_num]['pctchange'] <= 0.03:
            entry_flags.append(1)
        else:
            entry_flags.append(0)
    # Add in buffer None's
    for i in range(lookforward_num):
        entry_flags.append(None)
    return entry_flags

In [31]:
df['entry_flag'] = entry_flag(df, 6)
df['entry_flag'].value_counts()

0.0    4051
1.0    2389
2.0    1967
Name: entry_flag, dtype: int64

In [32]:
df.head(25)

,price_close,price_open,price_high,price_low,volume_traded,trades_count,pctchange,entry_flag
0,11.88950,11.87000,11.90000,11.85972,1076.171006,48,NaN,2.0
1,12.00000,11.89000,12.00000,11.89000,6184.443260,182,NaN,2.0
2,12.03000,12.00000,12.03000,11.93356,3391.487460,120,NaN,2.0
3,12.20000,12.02591,12.20000,12.00000,1590.240506,53,NaN,1.0
4,12.43100,12.20000,12.54799,12.20000,4801.330298,164,NaN,1.0
5,12.53369,12.39040,12.59809,12.31003,3566.448442,92,NaN,1.0
6,12.73950,12.53369,12.89950,12.48000,7815.423254,199,0.071492,0.0
7,12.59420,12.65747,12.79000,12.51000,2760.369206,65,0.049517,0.0
8,12.48257,12.59239,12.59840,12.44000,2267.837906,57,0.037620,0.0
9,12.46280,12.61000,12.61438,12.44005,3034.616506,87,0.021541,0.0


In [16]:
def create_sequence(df, n_input): 
    
    X,y = list(), list()
    
    start = n_input
    for _ in range(len(df)):
        # define the end of the input sequence
        end = start + n_input
        # ensure we have enough data for this instance 
        if end <= len(df) - n_input:
            df_slice = df.iloc[start:end]
            X.append(np.array(df_slice[['price_close', 'price_open', 'price_high', 'price_low', 'volume_traded', 'trades_count']]))
            y.append(df_slice['entry_flag'].tolist()[-1])
            print(X, y)
        # move along one time step
        start += 1
    return np.array(X), np.array(y).reshape(-1,1)

In [26]:
create_sequence(df[:20], 6)

[array([[  12.7395    ,   12.53369   ,   12.8995    ,   12.48      ,
        7815.42325418,  199.        ],
       [  12.5942    ,   12.65747   ,   12.79      ,   12.51      ,
        2760.36920641,   65.        ],
       [  12.48257   ,   12.59239   ,   12.5984    ,   12.44      ,
        2267.83790592,   57.        ],
       [  12.4628    ,   12.61      ,   12.61438   ,   12.44005   ,
        3034.616506  ,   87.        ],
       [  12.5226    ,   12.46501   ,   12.61349   ,   12.44005   ,
         481.65735478,   57.        ],
       [  12.55      ,   12.61349   ,   12.66407   ,   12.50017   ,
         702.78431946,  113.        ]])] [1.0]
[array([[  12.7395    ,   12.53369   ,   12.8995    ,   12.48      ,
        7815.42325418,  199.        ],
       [  12.5942    ,   12.65747   ,   12.79      ,   12.51      ,
        2760.36920641,   65.        ],
       [  12.48257   ,   12.59239   ,   12.5984    ,   12.44      ,
        2267.83790592,   57.        ],
       [  12.4628    ,   12

(array([[[   12.7395    ,    12.53369   ,    12.8995    ,    12.48      ,
           7815.42325418,   199.        ],
         [   12.5942    ,    12.65747   ,    12.79      ,    12.51      ,
           2760.36920641,    65.        ],
         [   12.48257   ,    12.59239   ,    12.5984    ,    12.44      ,
           2267.83790592,    57.        ],
         [   12.4628    ,    12.61      ,    12.61438   ,    12.44005   ,
           3034.616506  ,    87.        ],
         [   12.5226    ,    12.46501   ,    12.61349   ,    12.44005   ,
            481.65735478,    57.        ],
         [   12.55      ,    12.61349   ,    12.66407   ,    12.50017   ,
            702.78431946,   113.        ]],
 
        [[   12.5942    ,    12.65747   ,    12.79      ,    12.51      ,
           2760.36920641,    65.        ],
         [   12.48257   ,    12.59239   ,    12.5984    ,    12.44      ,
           2267.83790592,    57.        ],
         [   12.4628    ,    12.61      ,    12.61438   ,    